In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pymultinest import solve


# this may look different, depending on where you run this notebook :)
import lib.fitting as RM


#plt.rcParams.update({'font.size': 16})
plt.rc('font', size=12)          # controls default text sizes
#plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
#plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.width"] = 2
plt.rcParams["xtick.major.width"] = 2
plt.rcParams["xtick.major.size"] = 5
plt.rcParams["ytick.major.size"] = 5
plt.rcParams["xtick.minor.size"] = 3.5
plt.rcParams["ytick.minor.size"] = 3.5
#print(plt.rcParams.keys())
plt.rcParams["ytick.minor.width"] = 1
plt.rcParams["xtick.minor.width"] = 1
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.edgecolor'] = 'white'
plt.rcParams['legend.framealpha'] = 1

In [2]:
input_path = 'ExampleData/'

# fixed parameters 
aRs = 4.587
orbinc = 84.03 # in degrees
vsys = -24.83 # in km/s
u1 = 0.414
u2 = 0.15
gridsize = 50 # this produces a 100x100 grid for the stellar surface. The larger the grid, the slower the code 

# ground truth: This is what the fit should yield :) 
projected_obl = 70. # degrees
RpRs = 0.06
vsini = 100. # km/s
amp = 0.9
width = 4 # km/s

# load example data
RV = np.load(input_path+'RV.npy') 
phases = np.load(input_path+'orbital_phases.npy')
fake_data = np.load(input_path+'example_data.npy')
fake_data_err = np.load(input_path+'example_data_err.npy')

In [3]:
# Generating your flux grid based on limb darkening (quadratic) + your gridsize

xx, yy, flux_grid = RM.generate_flux_grid(
    u1=u1, 
    u2=u2, 
    gridsize=gridsize
)

In [ ]:
# plot example
plt.pcolormesh(RV, phases, fake_data)
plt.xlabel('RV [km/s]')
plt.ylabel('Orbital phase')

In [8]:
init_limits = [65., 80.],[0.05,0.07],[95.,105.],[-1, 1],[1., 10.]
parameters = ["lambda", "RpRs", "vsini", "amp", 'width']
output = 'results_pymultinest/test1'
mode='run' # can also be resume, run

if mode=='run':
    RM.remove_output(output)
    RM.write_json(output, parameters)

--- [INFO] removing results_pymultinest/test1phys_live.points
--- [INFO] removing results_pymultinest/test1resume.dat
--- [INFO] removing results_pymultinest/test1IS.points
--- [INFO] removing results_pymultinest/test1stats.dat
--- [INFO] removing results_pymultinest/test1stats.json
--- [INFO] removing results_pymultinest/test1IS.ptprob
--- [INFO] removing results_pymultinest/test1.txt
--- [INFO] removing results_pymultinest/test1summary.txt
--- [INFO] removing results_pymultinest/test1post_equal_weights.dat
--- [INFO] removing results_pymultinest/test1live.points
--- [INFO] removing results_pymultinest/test1corner.png
--- [INFO] removing results_pymultinest/test1params.json
--- [INFO] removing results_pymultinest/test1IS.iterinfo
--- [INFO] removing results_pymultinest/test1corner.pdf
--- [INFO] removing results_pymultinest/test1ev.dat
--- [INFO] Writing results_pymultinest/test1params.json 


In [ ]:
my_model = RM.DopplerShadow(
    # self,logL,prior,ndim,data,errdata,radial_velocity,phases,xx,yy,flux_grid,orbinc,aRs,vsys 
    logL = RM.compute_likelihood,
    prior = RM.uniform_prior,
    ndim = len(parameters),
    init_limits=init_limits,
    data = fake_data,
    errdata = fake_data_err,
    radial_velocity = RV,
    phases = phases, 
    xx = xx,
    yy = yy,
    flux_grid = flux_grid,
    orbinc = orbinc,
    aRs = aRs,
    vsys = vsys
)


result = solve(
    
    LogLikelihood = my_model.cpte_logL, # Gaussian!
    Prior = my_model.cpte_prior,
    n_dims = my_model.ndim, 
    outputfiles_basename = output,
    n_live_points=  500, 
    #dump_callback = True

)

print('evidence: %(logZ).1f +- %(logZerr).1f' % result)
print('parameter values:')

for name, col in zip(parameters, result['samples'].transpose()):
    print('%15s : %.6f +- %.6f' % (name, col.mean(), col.std()))

 MultiNest Warning: no resume file found, starting from scratch
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  500
 dimensionality =    5
 *****************************************************


In [ ]:
RM.multinest_marginals_corner(output)